In [1]:
import numpy as np
import time

## 1. Comparison

In [25]:
a = np.random.rand(1000000)
b = np.random.rand(1000000)

tic = time.time()
c = np.dot(a,b)
toc = time.time()

print(c)
print("Vectorization version: " + str(1000*(toc-tic)) + "ms")

c = 0
tic = time.time()
for i in range(1000000):
    c += a[i]*b[i]
toc = time.time()

print(c)
print("For loop version: " + str(1000*(toc-tic)) + "ms")

250126.50557589708
Vectorization version: 0.6661415100097656ms
250126.50557590416
For loop version: 331.38203620910645ms


## 2. Numpy Array Operations

In [26]:
print(a)
print(np.exp(a))
print(np.log(a))
print(np.abs(a))
print(np.max(a))  # 计算a的最大值
print(np.maximum(a,0))  # 计算a和0的最大值

[0.01866495 0.81352094 0.43856281 ... 0.43411878 0.78952949 0.90719659]
[1.01884023 2.25583669 1.55047729 ... 1.54360221 2.20235995 2.47736771]
[-3.98110784 -0.20638361 -0.82425224 ... -0.83443709 -0.23631809
 -0.0973961 ]
[0.01866495 0.81352094 0.43856281 ... 0.43411878 0.78952949 0.90719659]
0.9999999672336279
[0.01866495 0.81352094 0.43856281 ... 0.43411878 0.78952949 0.90719659]


## 3. Broadcasting in Python

In [28]:
A = np.array([[56.0, 0.0, 4.0, 68.0],
              [1.2, 104.0, 52.0, 8.0],
              [1.8, 135.0, 99.0, 0.9]])
print(A)

[[ 56.    0.    4.   68. ]
 [  1.2 104.   52.    8. ]
 [  1.8 135.   99.    0.9]]


In [29]:
cal = A.sum(axis=0)  # sum vertically; axis=1, sum horizontally
print(cal)

[ 59.  239.  155.   76.9]


In [32]:
percentage = 100*A/cal.reshape(1,4)  # reshape is unncessary, but it can ensure the shape of cal
print(percentage)

[[94.91525424  0.          2.58064516 88.42652796]
 [ 2.03389831 43.51464435 33.5483871  10.40312094]
 [ 3.05084746 56.48535565 63.87096774  1.17035111]]
